# Fits and characterizes final model for 26-week outcome

Gaylen Fronk  
May 27, 2024

### Set Up Environment

In [ ]:
study <- params$study
version <- params$version
cv <- params$cv
y_col_name <- params$y_col_name


Packages for script

In [ ]:

library(tidymodels)


── Attaching packages ────────────────────────────────────── tidymodels 1.2.0 ──

✔ broom        1.0.5      ✔ recipes      1.0.10
✔ dials        1.2.1      ✔ rsample      1.2.1 
✔ dplyr        1.1.4      ✔ tibble       3.2.1 
✔ ggplot2      3.5.1      ✔ tidyr        1.3.1 
✔ infer        1.0.7      ✔ tune         1.2.1 
✔ modeldata    1.3.0      ✔ workflows    1.1.4 
✔ parsnip      1.2.1      ✔ workflowsets 1.1.0 
✔ purrr        1.0.2      ✔ yardstick    1.3.1 

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard() masks scales::discard()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ recipes::step()  masks stats::step()
• Dig deeper into tidy modeling with R at https://www.tmwr.org

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ lubridate 1.9.3     ✔ stringr   1.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ readr::col_factor() masks scales::col_factor()
✖ purrr::discard()    masks scales::discard()
✖ dplyr::filter()     masks stats::filter()
✖ stringr::fixed()    masks recipes::fixed()
✖ dplyr::lag()        masks stats::lag()
✖ readr::spec()       masks yardstick::spec()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Attaching package: 'Matrix'

The following objects are masked from 'package:tidyr':

    expand, pack, unpack


Attaching package: 'probably'

The following objects are masked from 'package:base':

    as.factor, as.ordered

here() starts at C:/Users/gfronk/Documents/GitHub/study_match

ℹ SHA-1 hash of file is "c045eee2655a18dc85e715b78182f176327358a7"

ℹ SHA-1 hash of file is "b58b9e0adc05e89609ee8bc6f20c39375b262fe6"

Handle conflicts

In [ ]:

options(conflicts.policy = "depends.ok")


Absolute paths

In [ ]:
switch (Sys.info()[['sysname']],
        # PC paths
        Windows = {
          path_input <- stringr::str_c("P:/studydata/match/chtc/", 
                                       y_col_name)
          path_models <- stringr::str_c("P:/studydata/match/models/", 
                                        y_col_name)},
        
        # IOS paths
        Darwin = {
          path_input <- stringr::str_c("/Volumes/private/studydata/match/chtc/", 
                                       y_col_name)
          path_models <- stringr::str_c("/Volumes/private/studydata/match/models/", 
                                        y_col_name)},
        
        # Linux paths
        Linux = {
          path_input <- stringr::str_c("~/mnt/private/studydata/match/chtc/", 
                                       y_col_name)
          path_models <- stringr::str_c("~/mnt/private/studydata/match/models/", 
                                        y_col_name)}
)


Chunk Defaults

In [ ]:

knitr::opts_chunk$set(attr.output='style="max-height: 500px;"')

options(tibble.width = Inf)
options(tibble.print_max = Inf)


### Read in best configuration

Selected in 1_metrics_inner (k-fold CV)

In [ ]:
best_config <- read_csv(file.path(path_models, 
                                  str_c("best_config_none_", version, ".csv")),
                        show_col_types = FALSE)

glimpse(best_config)


Rows: 1
Columns: 14
$ n_jobs       <dbl> 10
$ algorithm    <chr> "glmnet"
$ feature_set  <chr> "scale_ordinal"
$ hp1          <dbl> 0.3
$ hp2          <dbl> 0.05645159
$ hp3          <lgl> NA
$ resample     <chr> "none"
$ accuracy     <dbl> 0.766055
$ bal_accuracy <dbl> 0.5
$ roc_auc      <dbl> 0.6727607
$ sens         <dbl> 0
$ spec         <dbl> 1
$ ppv          <lgl> NA
$ npv          <dbl> 0.7685185

### Fit best model in full dataset

**NOTE**: This model will only be used for 1) examining parameter estimates, and 2) ultimate dissemination.

In [ ]:
batch_names <- list.dirs(path_input, full.names = FALSE, recursive = FALSE) 

batch_name <- batch_names[str_detect(batch_names, "train") & 
                            str_detect(batch_names, cv) &
                            str_detect(batch_names, version) &
                            str_detect(batch_names, best_config$algorithm)] 

path_batch <- file.path(path_input, batch_name)
source(file.path(path_batch, "input", "training_controls.R"))

d <- read_csv(file.path(path_batch, "input", "data_trn.csv"), 
              show_col_types = FALSE) 

d_outcomes <- d |> 
  select(subid, ends_with("outcome") & contains("hybrid"))

d <- format_data(d) 

rec <- build_recipe(d = d, config = best_config)

rec_prepped <- rec |> 
  prep(training = d, strings_as_factors = FALSE)

feat_all <- rec_prepped |> 
  bake(new_data = NULL)

model_best <- fit_best_model(best_model = best_config, 
                             feat = feat_all, 
                             ml_mode = "classification")


### Model coefficients

**NOTE**: Coefficients are naturally inverted (i.e., positive class \[abstinence\] treated as first \[vs. second\] class). Here, we multiply coefficients by -1 to align the direction of coefficients with the rest of our analyses. Once flipped (i.e., as they appear below)…

A *positive coefficient* indicates that increases in the feature *increase* the likelihood of abstinence. For example, increases in motivation to quit (“motive_quit_order”) *increase* the likelihood of abstinence.

A *negative coefficient* indicates that increases in the feature *decrease* the likelihood of smoking. For example, increases in carbon monoxide (“co”) *decrease* the likelihood of abstinence.

In [ ]:
model_tidy <- tidy(model_best)


Loaded glmnet 4.1-8

In [ ]:
retained_vars <- model_tidy |> 
  mutate(estimate = estimate * -1) |> 
  filter(abs(estimate) > 0) |> 
  select(-penalty) |> 
  arrange(desc(abs(estimate)))

print_kbl(retained_vars, digits = 4)


7.74% of features were retained by the glmnet solution (38 features of a total considered 491 features).

In [ ]:
retained_vars_tx <- retained_vars |> 
  filter(str_detect(term, "treatment_")) |> 
  arrange(desc(abs(estimate))) 

print_kbl(retained_vars_tx, digits = 4)


26.32% of the retained features were treatment interaction features (10 treatment features of 38 retained features). Of the 354 available treatment interaction features, 3% were retained.

### Calculate and calibrate probabilities

Make triplicate dataset

In [ ]:
d_patch <- d |> 
  mutate(treatment = "patch") 

d_combo <- d |> 
  mutate(treatment = "combo_nrt")

d_varen <- d |> 
  mutate(treatment = "varenicline")

d_trip <- bind_rows(d_patch, d_combo) |> 
  bind_rows(d_varen) |> 
  mutate(treatment = factor(treatment, 
                            levels = c(
                              "patch",
                              "varenicline",
                              "combo_nrt")))


Make function to: 1) hold out each sub once, 2) fit model with remaining 1085 subs, 3) get 3 raw predictions for held-out sub, 4) get calibrated predictions for held-out sub

In [ ]:
get_triple_probs <- function(i_sub, best_config, d, d_trip){
  
  # hold out single subject
  d_out <- d_trip |> 
    filter(subid == i_sub)
  
  d_in <- d |> 
    filter(subid != i_sub)
  
  # prep recipe 
  rec <- build_recipe(d = d_in, config = best_config)
  rec_prepped <- rec |> 
    prep(training = d_in, strings_as_factors = FALSE)
  
  # bake feat_in and feat_out
  feat_in <- rec_prepped |> 
    bake(new_data = NULL)
  
  feat_out <- rec_prepped |> 
    bake(new_data = d_out)
  
  # fit model
  model_best <- fit_best_model(best_config, feat = feat_in,
                               "classification")
  
  # raw (uncalibrated) predictions
  preds_prob <- predict(model_best, feat_out,
                        type = "prob")
  
  # fit calibration model
  set.seed(2468)
  cal_split <- d_in |> 
    initial_split(prop = 3/4, strata = y)
  d_cal_in <- training(cal_split) 
  d_cal_out <- testing(cal_split)
  
  rec_cal_prepped <- rec |> 
    prep(training = d_cal_in, strings_as_factors = FALSE)
  
  feat_cal_in <- rec_cal_prepped |> 
    bake(new_data = NULL) 
  
  feat_cal_out <- rec_cal_prepped |> 
    bake(new_data = d_cal_out) 
  
  model_cal <- fit_best_model(best_config, feat = feat_cal_in, "classification")
  
  # beta calibration
  beta <- predict(model_cal, feat_cal_out,
                  type = "prob") |>
    mutate(truth = feat_cal_out$y) |>
    cal_estimate_beta(truth = truth,
                      estimate = dplyr::starts_with(".pred_"),
                      smooth = TRUE)
  preds_prob_beta <- preds_prob |>
    cal_apply(beta)
  
  # iso calibration
  iso <- predict(model_cal, feat_cal_out,
                 type = "prob") |>
    mutate(truth = feat_cal_out$y) |>
    cal_estimate_isotonic(truth = truth,
                          estimate = dplyr::starts_with(".pred_"))
  preds_prob_iso <- preds_prob |>
    cal_apply(iso)
  
  # logistic calibration
  logi <- predict(model_cal, feat_cal_out,
                  type = "prob") |>
    mutate(truth = feat_cal_out$y) |>
    cal_estimate_logistic(truth = truth,
                          estimate = dplyr::starts_with(".pred_"),
                          smooth = TRUE)
  preds_prob_logi <- preds_prob |>
    cal_apply(logi)
  
  # combine raw and calibrated probs
  probs <- tibble(subid = d_out$subid,
                  tx = d_out$treatment,
                  prob_raw = preds_prob[[str_c(".pred_", y_level_pos)]],
                  prob_beta = preds_prob_beta[[str_c(".pred_", y_level_pos)]],
                  prob_iso = preds_prob_iso[[str_c(".pred_", y_level_pos)]],
                  prob_logi = preds_prob_logi[[str_c(".pred_", y_level_pos)]]) 
}


Map over participants so that each is held-out once

In [ ]:
if(file.exists(file.path(path_models, 
                         str_c("trip_probs_none_", version, ".csv")))) {
  all_probs <- read_csv(file.path(path_models, 
                                  str_c("trip_probs_none_", version, ".csv")),
                        show_col_types = FALSE)
} else {
  all_probs <- d$subid |> 
  map(\(i_sub) get_triple_probs(i_sub, best_config, d, d_trip)) |> 
  list_rbind()
  
  all_probs |> 
    write_csv(file.path(path_models, 
                        str_c("trip_probs_none_", version, ".csv")))
}


### Make AIM 2 Dataset

Pivot probabilities into wide format & select only raw probability (based on calibration validity check below)

In [ ]:
raw_probs_wide <- all_probs |> 
  select(subid, tx, prob_raw) |> 
  pivot_wider(names_prefix = "prob_",
              names_from = tx,
              values_from = prob_raw)

glimpse(raw_probs_wide)


Rows: 1,086
Columns: 4
$ subid            <dbl> 20010, 20015, 20030, 20049, 20051, 20072, 20077, 2008…
$ prob_patch       <dbl> 0.1514963, 0.2501455, 0.1857522, 0.2281568, 0.2423632…
$ prob_combo_nrt   <dbl> 0.1910656, 0.2543865, 0.2146872, 0.2319471, 0.2423632…
$ prob_varenicline <dbl> 0.1168724, 0.2462586, 0.1483799, 0.2380323, 0.2437960…

Join with d & create new variables

In [ ]:
set.seed(52592)
d_aim_2 <- d |> 
  select(subid, tx_rct = treatment, outcome_rct_wk26 = y) |> 
  left_join(raw_probs_wide, by = "subid") |> 
  mutate(tx_best = case_when(
    prob_patch > prob_combo_nrt & prob_patch > prob_varenicline ~ "patch",
    prob_combo_nrt > prob_patch & prob_combo_nrt > prob_varenicline ~ "combo_nrt",
    prob_varenicline > prob_patch & prob_varenicline > prob_combo_nrt ~ "varenicline",
    TRUE ~ NA_character_
  )) |> 
  mutate(tx_best = if_else(is.na(tx_best), case_when(
    prob_patch == prob_combo_nrt ~ "combo_nrt",
    prob_patch == prob_varenicline ~ "varenicline",
    prob_combo_nrt == prob_varenicline ~ sample(c("combo_nrt",
                                                  "varenicline"), 1),
    TRUE ~ NA_character_
  ), tx_best)) |> 
  mutate(prob_best = case_when(
    tx_best == "patch" ~ prob_patch,
    tx_best == "combo_nrt" ~ prob_combo_nrt,
    tx_best == "varenicline" ~ prob_varenicline,
    TRUE ~ NA_real_
  )) |> 
  mutate(tx_best = factor(tx_best, 
                          levels = c(
                            "patch",
                            "varenicline",
                            "combo_nrt"))) |> 
  mutate(tx_match = if_else(tx_best == tx_rct, TRUE, FALSE)) |> 
  left_join(d_outcomes, by = "subid") |> 
  select(-pp_hybrid_wk26_outcome, -pp_hybrid_wk1_outcome, -pp_hybrid_yr3_outcome, -pp_hybrid_wk52_outcome) |> # already in there from d
  rename(outcome_rct_wk12 = pp_hybrid_wk12_outcome,
         outcome_rct_wk4 = pp_hybrid_wk4_outcome) |> 
  relocate(subid, tx_rct, tx_best, tx_match,
           prob_best, starts_with("outcome"), starts_with("prob"))

glimpse(d_aim_2)


Rows: 1,086
Columns: 11
$ subid            <dbl> 20010, 20015, 20030, 20049, 20051, 20072, 20077, 2008…
$ tx_rct           <fct> patch, combo_nrt, patch, varenicline, patch, combo_nr…
$ tx_best          <fct> combo_nrt, combo_nrt, combo_nrt, varenicline, varenic…
$ tx_match         <lgl> FALSE, TRUE, FALSE, TRUE, FALSE, FALSE, FALSE, FALSE,…
$ prob_best        <dbl> 0.1910656, 0.2543865, 0.2146872, 0.2380323, 0.2437960…
$ outcome_rct_wk26 <fct> smoking, smoking, smoking, abstinent, smoking, smokin…
$ outcome_rct_wk4  <chr> "smoking", "smoking", "smoking", "smoking", "smoking"…
$ outcome_rct_wk12 <chr> "smoking", "smoking", "smoking", "abstinent", "smokin…
$ prob_patch       <dbl> 0.1514963, 0.2501455, 0.1857522, 0.2281568, 0.2423632…
$ prob_combo_nrt   <dbl> 0.1910656, 0.2543865, 0.2146872, 0.2319471, 0.2423632…
$ prob_varenicline <dbl> 0.1168724, 0.2462586, 0.1483799, 0.2380323, 0.2437960…

Quick EDA checks

In [ ]:
# confirm logic worked as expected
head(d_aim_2)


# A tibble: 6 × 11
  subid tx_rct      tx_best     tx_match prob_best outcome_rct_wk26
  <dbl> <fct>       <fct>       <lgl>        <dbl> <fct>           
1 20010 patch       combo_nrt   FALSE        0.191 smoking         
2 20015 combo_nrt   combo_nrt   TRUE         0.254 smoking         
3 20030 patch       combo_nrt   FALSE        0.215 smoking         
4 20049 varenicline varenicline TRUE         0.238 abstinent       
5 20051 patch       varenicline FALSE        0.244 smoking         
6 20072 combo_nrt   varenicline FALSE        0.268 smoking         
  outcome_rct_wk4 outcome_rct_wk12 prob_patch prob_combo_nrt prob_varenicline
  <chr>           <chr>                 <dbl>          <dbl>            <dbl>
1 smoking         smoking               0.151          0.191            0.117
2 smoking         smoking               0.250          0.254            0.246
3 smoking         smoking               0.186          0.215            0.148
4 smoking         abstinent             0.228  

# A tibble: 2 × 3
  tx_best         n  prop
  <fct>       <int> <dbl>
1 varenicline   285 0.262
2 combo_nrt     801 0.738

# A tibble: 2 × 3
  tx_match     n  prop
  <lgl>    <int> <dbl>
1 FALSE      648 0.597
2 TRUE       438 0.403

# A tibble: 5 × 4
# Groups:   tx_rct [3]
  tx_rct      tx_match     n  prop
  <fct>       <lgl>    <int> <dbl>
1 patch       FALSE      241 1    
2 varenicline FALSE      307 0.724
3 varenicline TRUE       117 0.276
4 combo_nrt   FALSE      100 0.238
5 combo_nrt   TRUE       321 0.762

Write out

In [ ]:
d_aim_2 |> 
  write_csv(file.path(path_models, str_c("aim_2_", version, "_", 
                                   y_col_name, ".csv")))

# d_aim_2 |> 
#   write_csv(here("/objects", str_c("aim_2_", version, "_", 
#                                    y_col_name, ".csv")))


### Validity check: Model Calibration

Make wide format for all calibrated probabilities

In [ ]:
all_probs_wide <- all_probs |> 
  rename_with(~ str_replace(.x, "prob_", ""), .cols = starts_with("prob_")) |> 
  pivot_wider(
    names_from = tx,
    values_from = c(raw, beta, iso, logi),
    names_glue = "{.value}_{tx}"
  )


Set up validity check data

In [ ]:
d_valid <- d_aim_2 |> 
  select(subid, tx_rct, outcome_rct_wk26) |> 
  left_join(all_probs_wide, by = "subid") |> 
  mutate(beta = case_when(
    tx_rct == "patch" ~ beta_patch,
    tx_rct == "combo_nrt" ~ beta_combo_nrt,
    tx_rct == "varenicline" ~ beta_varenicline,
    TRUE ~ NA_real_
  ),
  raw = case_when(
    tx_rct == "patch" ~ raw_patch,
    tx_rct == "combo_nrt" ~ raw_combo_nrt,
    tx_rct == "varenicline" ~ raw_varenicline,
    TRUE ~ NA_real_
  ),
  isotonic = case_when(
    tx_rct == "patch" ~ iso_patch,
    tx_rct == "combo_nrt" ~ iso_combo_nrt,
    tx_rct == "varenicline" ~ iso_varenicline,
    TRUE ~ NA_real_
  ),
  logistic = case_when(
    tx_rct == "patch" ~ logi_patch,
    tx_rct == "combo_nrt" ~ logi_combo_nrt,
    tx_rct == "varenicline" ~ logi_varenicline,
    TRUE ~ NA_real_
  )) |> 
  select(subid, outcome_rct_wk26, beta, raw, isotonic, logistic) 


#### Comparison of means

Compare mean abstinence from RCT to predicted probabilities for RCT tx across calibrations

In [ ]:
d_valid |> 
  mutate(outcome_rct = if_else(outcome_rct_wk26 == "abstinent", 1, 0)) |> 
  summarize(mean_rct = mean(outcome_rct),
            mean_beta = mean(beta),
            mean_iso = mean(isotonic),
            mean_logi = mean(logistic),
            mean_raw = mean(raw))


# A tibble: 1 × 5
  mean_rct mean_beta mean_iso mean_logi mean_raw
     <dbl>     <dbl>    <dbl>     <dbl>    <dbl>
1    0.247     0.255    0.169     0.255    0.246

Raw probabilities appear to match the outcome best.

#### Plots

Pivot data longer for required format

In [ ]:
d_plot <- d_valid |> 
  pivot_longer(
    cols = c(beta, raw, isotonic, logistic),
    names_to = "method",
    values_to = ".pred_abstinent"
  )


Make plots by calibration method

In [ ]:
d_plot |> 
  cal_plot_breaks(truth = outcome_rct_wk26, 
                  estimate = .pred_abstinent,
                  .by = method)


Logistic and raw probabilities follow the diagonal line for the longest. Both jump off the line around the midpoint, with logistic probabilities jumping up, and raw probabilities jumping in both directions.

#### Brier score

Beta calibration

In [ ]:
brier_class_vec(truth = d_valid$outcome_rct_wk26,
                estimate = d_valid$beta)


[1] 0.1792077

Logistic calibration

In [ ]:
brier_class_vec(truth = d_valid$outcome_rct_wk26,
                estimate = d_valid$logistic)


[1] 0.1791732

Isotonic calibration

In [ ]:
brier_class_vec(truth = d_valid$outcome_rct_wk26,
                estimate = d_valid$isotonic)


[1] 0.1861293

Raw calibration

In [ ]:
brier_class_vec(truth = d_valid$outcome_rct_wk26,
                estimate = d_valid$raw)


[1] 0.1759518